# **Auxiliary code for recommendation system**

### Pre-process features and calculation of recommendation score

***Created by Gaurav Lodhi***

In [ ]:
#################################
"""Recipe Recommendataion System"""
#################################

import ast
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer

**Pre-processing, Normalization and saving the modified dataframe to be used directly in the GUI.**

In [ ]:
lmtzr = WordNetLemmatizer()

# Preprocessing
data = pd.read_csv("recipes.csv")
data2 = data.copy()
temp = []
val = 1.0
for y in data['Calories']:
    try:
        val = float(y)
    except:
        y = y.replace(",", "")
        val = float(y)
    temp.append(val)
data['Calories'] = temp
cols = ['Ingredients', 'Cooking instructions', 'Rating', 'Lookup Ingredients']
for x in cols:
    l = []
    for y in data[x]:
        try:
            res = ast.literal_eval(y)
            if x == 'Lookup Ingredients':
                res = [lmtzr.lemmatize(word) for word in res]
        except:
            print(y)
        l.append(res)
    data[x] = l
cols_to_norm = ['Calories', 'Fat', 'Cholesterol', 'Carbs', 'Fiber', 'Protein']
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
data.to_csv("recommendation_dataframe.csv", index=False)

## **Method to calculate recommendation score for the specified meal type and list of ingredients**

In [ ]:
def recommend(data_type_recipes, ingredients, k):
    com_val = {}
    for i in range(len(data_type_recipes)):
        neg_factors = np.mean([data_type_recipes['Calories'].iloc[i], data_type_recipes['Fat'].iloc[i],
                               data_type_recipes['Cholesterol'].iloc[i]])
        pos_factors = np.mean([data_type_recipes['Rating Score'].iloc[i], data_type_recipes['Carbs'].iloc[i],
                               data_type_recipes['Fiber'].iloc[i], data_type_recipes['Protein'].iloc[i]])
        com_val[data_type_recipes['Recipe ID'].iloc[i]] = len(
            set(ingredients) & set(data_type_recipes['Lookup Ingredients'].iloc[i])) * (pos_factors - neg_factors)
    return sorted(com_val, key=com_val.get, reverse=True)[:k]

**Testing functionality before integrating it with the GUI screen**

In [ ]:
#######################################
# input
k = 10
type_of_food = 'Dinner'
ingredients = ['chicken', 'beef']

# lemmatizer on input
ingredients = [lmtzr.lemmatize(word) for word in ingredients]

data_type_recipes = data[data['Meal'] == type_of_food]
data_type_recipes = data_type_recipes.dropna()
recommendations = recommend(data_type_recipes, ingredients, k)
recommendation_urls = []
recommend_recipes = []
print(recommendations)
for x in recommendations:
    val = data_type_recipes.loc[data_type_recipes['Recipe ID'] == x, ['URL', 'Title']]
    recommendation_urls.append(list(val['URL'])[0])
    recommend_recipes.append(list(val['Title'])[0])

print("Top K recommended Recipes:\n", recommend_recipes, '\n')
# print("Top K Recommended Recipes Urls:\n", recommendation_urls, '\n')